## Presentation Types with Hidden Types
## Also Network Visualization with Louvain Communities

* March 2022 version

    * Uses getDistance to identify `close matches` with side-by-side comparison of soggetti.  With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same.  These are reported as "flexed entries" in a separate column.

    * Labels Fuga, PEn, and ID according to time intervals.  
    * If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
    * Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
    * Counts number of entries
    * Provides offset and measure/beat locations
    * Sorts all presentation types by the order in which they appear in the piece
    * Reports voice names of the entries, in order of their appearance
    * Omits singleton soggetti (just one entry of a given motive in isolation)
    
    ALSO
    
    * Finds "hidden" types within a longer Fuga.  That is, if a 5-voice fuga also contains a PEN, it will label both of these as separate presentation type, along with all the relevant data noted above.

In [1]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


#### The following are special functions used by the classifier.  Don't change them.

## Load one Piece Here

* Note that you can load from CRIM, or put a file in the **Music_Files** folder in the Notebook.

In [2]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# just add the CRIM Piece ID here
mei_file = 'CRIM_Mass_0019_2.mei'


url = git_prefix + mei_file
# piece = importScore('Music_Files/Senfl_Ave_forCRIM.mei_msg.mei')
piece = importScore(url)
# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')

print(piece.metadata)

Successfully imported https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/CRIM_Mass_0019_2.mei
{'title': 'Missa Veni sponsa Christi: Gloria', 'composer': 'Giovanni Pierluigi da Palestrina'}


## Run the Classifier Here

- set the length of the soggetti with `melodic_ngram_length`
- set the maximum difference between similar soggetti with `edit_distance_threshold`
- for chromatic vs diatonic, compound, and directed data in soggetti, see `interval_settings`
- to include all the hidden PENs and IDS (those found within longer Fugas, use `include_hidden_types == True`.  
- for faster (and simpler) listing of points of imitation without hidden forms, use `include_hidden_types == False`



In [3]:
include_hidden_types = True
combine_unisons = True
melodic_ngram_length = 4
edit_distance_threshold = 1
nr = piece.getNoteRest(combineUnisons=combine_unisons)
dur = piece.getDuration(df=nr)
mel = piece.getMelodic(df=nr, kind='d', end=False)
dur_ng = piece.getNgrams(df=dur, n=melodic_ngram_length)
mel_ng = piece.getNgrams(df=mel, n=melodic_ngram_length)
entries = piece.getEntries(mel_ng)
output = classify_entries_as_presentation_types(piece, nr, dur_ng, entries, edit_distance_threshold, include_hidden_types)


#### Below Find Source Code and Explanation of the Method

In [4]:
output

,index,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,0,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Gloria,0.0,"[1/1.0, 4/3.0]",[P-8],"[0.0, 28.0]","[-3, 3, 2, -2, -3, 2, 2, -2]",[28.0],"[Cantus, Tenor]",FUGA,2,True
1,1,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Gloria,4.0,"[1/3.0, 5/1.0]",[P-8],"[4.0, 32.0]","[-3, 2, 2, -5]",[28.0],"[Altus, Bassus]",FUGA,2,False
2,2,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Gloria,50.0,"[7/2.0, 7/4.0, 10/3.0, 12/3.0]","[P5, P-12, P8]","[50.0, 54.0, 76.0, 92.0]","[-2, -3, 3, -2]","[4.0, 22.0, 16.0]","[Altus, Cantus, Bassus, Altus]",FUGA,4,False
3,3,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Gloria,58.0,"[8/2.0, 11/3.0]",[P5],"[58.0, 84.0]","[-2, -3, 2, 2, -2, -3, 3, 2]",[26.0],"[Tenor, Cantus]",FUGA,2,True
4,4,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Gloria,236.0,"[30/3.0, 34/4.0, 35/2.0, 37/4.0, 39/2.0, 40/2.0]","[P-4, P5, P-12, P5, P8]","[236.0, 270.0, 274.0, 294.0, 306.0, 314.0]","[-2, 2, 3, -2]","[34.0, 4.0, 20.0, 12.0, 8.0]","[Cantus, Altus, Cantus, Bassus, Tenor, Cantus]",FUGA,6,False
5,5,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Gloria,274.0,"[35/2.0, 37/4.0, 40/2.0]","[P-12, P12]","[274.0, 294.0, 314.0]","[-2, 2, 3, -2]","[20.0, 20.0]","[Cantus, Bassus, Cantus]",PEN,3,False
6,6,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Gloria,352.0,"[44/1.0, 45/1.0, 46/1.0, 47/1.0]","[P8, P-12, P8]","[352.0, 360.0, 368.0, 376.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 8.0, 8.0]","[Tenor, Cantus, Bassus, Altus]",PEN,4,True
7,7,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Gloria,352.0,"[44/1.0, 46/1.0, 60/2.0, 62/2.0]","[P-5, P8, P1]","[352.0, 368.0, 482.0, 498.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[16.0, 114.0, 16.0]","[Tenor, Bassus, Altus, Tenor]",ID,4,True
8,8,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Gloria,352.0,"[44/1.0, 45/1.0, 46/1.0, 47/1.0, 50/2.0, 54/1....","[P8, P-12, P8, M3, M-10, P1, P8, P1, P1]","[352.0, 360.0, 368.0, 376.0, 402.0, 402.0, 432...","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 8.0, 8.0, 26.0, 0.0, 30.0, 10.0, 40.0, 1...","[Tenor, Cantus, Bassus, Altus, Cantus, Bassus,...",FUGA,10,True
9,9,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Gloria,352.0,"[44/1.0, 45/1.0, 46/1.0]","[P8, P-12]","[352.0, 360.0, 368.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 8.0]","[Tenor, Cantus, Bassus]",PEN,3,True


### Run Classifier on Several Pieces at Once

Results are combined into a single dataframe

In [13]:


git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

# piece = importScore('Music_Files/CRIM_Mass_0007_4.mei')
piece_list =  ['CRIM_Mass_0005_3.mei',
             'CRIM_Mass_0005_4.mei',
             'CRIM_Mass_0005_5.mei',
             'CRIM_Model_0001.mei',
             'CRIM_Mass_0002_1.mei',
             'CRIM_Mass_0002_2.mei',
             'CRIM_Mass_0002_3.mei',
             'CRIM_Mass_0002_4.mei',
             'CRIM_Mass_0002_5.mei',
             'CRIM_Model_0015.mei',
             'CRIM_Mass_0013_1.mei',
             'CRIM_Mass_0013_2.mei',
             'CRIM_Mass_0013_3.mei',
             'CRIM_Mass_0013_4.mei',
             'CRIM_Mass_0013_5.mei',
             'CRIM_Model_0019.mei',
             'CRIM_Mass_0019_1.mei',
             'CRIM_Mass_0019_2.mei',
             'CRIM_Mass_0019_3.mei',
             'CRIM_Mass_0019_4.mei',
             'CRIM_Mass_0019_5.mei']


In [2]:
git_prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

piece_list = ['CRIM_Model_0019.mei',
             'CRIM_Mass_0019_1.mei',
             'CRIM_Mass_0019_2.mei',
             'CRIM_Mass_0019_3.mei',
             'CRIM_Mass_0019_4.mei',
             'CRIM_Mass_0019_5.mei']

In [14]:
include_hidden_types = False
melodic_ngram_length = 4
edit_distance_threshold = 0
final = pd.DataFrame()
for work in piece_list:
    url = git_prefix + work
    piece = importScore(url)   
    nr = piece.getNoteRest(combineUnisons=True)
    dur = piece.getDuration(df=nr)
    dur_ng = piece.getNgrams(df=dur, n=melodic_ngram_length)
    mel = piece.getMelodic(df=nr)
    mel_ng = piece.getMelodicEntries(interval_settings=('d', True, True), n=melodic_ngram_length)
    output = classify_entries_as_presentation_types(piece, nr, dur_ng, mel_ng, edit_distance_threshold, include_hidden_types)
    final = final.append(output, ignore_index=True)
final

Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.
Memoized piece detected.


,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries
0,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0, 11/3.0, 14/1.0, 1...","[P-5, P-4, P-5, P8, P5, P-8]","[0.0, 8.0, 40.0, 48.0, 84.0, 104.0, 128.0]","[-3, 3, 2, -2]","[8.0, 32.0, 8.0, 36.0, 20.0, 24.0]","[Cantus, Altus, Tenor, Bassus, Altus, Cantus, ...",FUGA,7,False
1,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,150.0,"[19/4.0, 21/4.0, 22/4.0, 25/4.0, 27/4.0, 29/4....","[P8, P-5, P-4, P-5, P8, P8, P-11]","[150.0, 166.0, 174.0, 198.0, 214.0, 230.0, 250...","[-3, 2, 2, -2]","[16.0, 8.0, 24.0, 16.0, 16.0, 20.0, 24.0]","[Tenor, Cantus, Altus, Tenor, Bassus, Altus, C...",FUGA,8,False
2,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,298.0,"[38/2.0, 38/4.0, 40/2.0, 42/2.0, 44/1.0, 46/2....","[P-5, P8, P-12, P8, P8, P-11]","[298.0, 302.0, 314.0, 330.0, 344.0, 362.0, 382.0]","[1, 1, -3, 3]","[4.0, 12.0, 16.0, 14.0, 18.0, 20.0]","[Altus, Tenor, Cantus, Bassus, Altus, Cantus, ...",FUGA,7,False
3,"Palestrina, Giovanni Pierluigi da",Veni sponsa Christi,394.0,"[50/2.0, 51/4.0, 53/2.0, 55/2.0, 56/4.0, 59/1....","[P5, P-8, P4, P5, P-8, P8, P-4, P-5]","[394.0, 406.0, 418.0, 434.0, 446.0, 464.0, 486...","[-2, 2, 1, 3]","[12.0, 12.0, 16.0, 12.0, 18.0, 22.0, 4.0, 16.0]","[Altus, Cantus, Bassus, Tenor, Altus, Bassus, ...",FUGA,9,False
4,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Kyrie,0.0,"[1/1.0, 2/1.0, 6/1.0, 7/1.0]","[P-5, P-4, P-5]","[0.0, 8.0, 40.0, 48.0]","[-3, 3, 2, -2]","[8.0, 32.0, 8.0]","[Cantus, Altus, Tenor, Bassus]",ID,4,False
...,...,...,...,...,...,...,...,...,...,...,...,...
56,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,280.0,"[35/1.0, 38/1.0]",[P-8],"[280.0, 304.0]","[-3, 3, 2, -2]",[24.0],"[Cantus, Quinta Pars]",FUGA,2,False
57,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,332.0,"[41/3.0, 43/2.0, 44/3.0, 46/2.0, 48/4.0]","[P-5, P-4, P4, P-8]","[332.0, 346.0, 356.0, 370.0, 390.0]","[1, -3, 1, 2]","[14.0, 10.0, 14.0, 20.0]","[Cantus, Altus, Quinta Pars, Tenor, Bassus]",FUGA,5,False
58,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,404.0,"[50/3.0, 53/3.0]",[P-8],"[404.0, 428.0]","[1, 1, -3, 3]",[24.0],"[Cantus, Quinta Pars]",FUGA,2,False
59,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,446.0,"[55/4.0, 56/4.0, 59/4.0, 61/4.0, 64/4.0]","[P1, P-8, P11, P-8]","[446.0, 454.0, 478.0, 494.0, 518.0]","[-2, 2, 1, 3]","[8.0, 24.0, 16.0, 24.0]","[Altus, Cantus, Quinta Pars, Cantus, Quinta Pars]",FUGA,5,False


In [15]:
final.to_csv("saved_csv/Mass_0019_PTsimple.csv")

In [16]:
final["MINT"] = final["Melodic_Entry_Intervals"].apply(joiner)
final["TINT"] = final["Time_Entry_Intervals"].apply(joiner)

final['SOG'] = final['Soggetti'].apply(clean_melody)
final['ALL'] = final["MINT"] + '_' + final["TINT"] + '_' + final['SOG']

final["ALL_INT"] = final["MINT"] + '_' + final["TINT"]
final["ALL_SOG"] = final["MINT"] + '_' + final["SOG"]
final

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,MINT,TINT,SOG,ALL,ALL_INT,ALL_SOG
0,Antoine de Févin,Missa Ave Maria: Credo,0.0,"[1/1.0, 3/3.0]",[P8],"[0.0, 20.0]","[4, 1, 1, 2, 4, 1, 2, 2]",[20.0],"[Altus, Sup[erius]]",FUGA,2,True,P8,20.0,4_1_1_2,P8_20.0_4_1_1_2,P8_20.0,P8_4_1_1_2
1,Antoine de Févin,Missa Ave Maria: Credo,30.0,"[4/4.0, 7/2.0]",[P8],"[30.0, 50.0]","[4, 1, -2, -2]",[20.0],"[Altus, Sup[erius]]",FUGA,2,False,P8,20.0,4_1_-2_-2,P8_20.0_4_1_-2_-2,P8_20.0,P8_4_1_-2_-2
2,Antoine de Févin,Missa Ave Maria: Credo,90.0,"[12/2.0, 12/4.0]",[m6],"[90.0, 94.0]","[1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1]",[4.0],"[Altus, Sup[erius]]",FUGA,2,True,m6,4.0,1_1_3_1,m6_4.0_1_1_3_1,m6_4.0,m6_1_1_3_1
3,Antoine de Févin,Missa Ave Maria: Credo,118.0,"[15/4.0, 16/2.0]",[P5],"[118.0, 122.0]","[2, 2, 2, 2, 2, 2, 2, 1]",[4.0],"[Altus, Sup[erius]]",FUGA,2,True,P5,4.0,2_2_2_2,P5_4.0_2_2_2_2,P5_4.0,P5_2_2_2_2
4,Antoine de Févin,Missa Ave Maria: Credo,156.0,"[20/3.0, 27/2.0, 27/4.0]","[m-7, P5]","[156.0, 210.0, 214.0]","[1, 2, 2, -2, 2, 2, 2, -2]","[54.0, 4.0]","[Tenor, Bassus, Tenor]",FUGA,3,True,m-7_P5,54.0_4.0,1_2_2_-2,m-7_P5_54.0_4.0_1_2_2_-2,m-7_P5_54.0_4.0,m-7_P5_1_2_2_-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,0.0,"[1/1.0, 2/1.0, 5/3.0, 7/3.0, 8/4.0]","[P-5, P-4, P-5, P8]","[0.0, 8.0, 36.0, 52.0, 62.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 28.0, 16.0, 10.0]","[Cantus, Altus, Tenor, Bassus, Altus]",FUGA,5,True,P-5_P-4_P-5_P8,8.0_28.0_16.0_10.0,-3_3_2_-2,P-5_P-4_P-5_P8_8.0_28.0_16.0_10.0_-3_3_2_-2,P-5_P-4_P-5_P8_8.0_28.0_16.0_10.0,P-5_P-4_P-5_P8_-3_3_2_-2
241,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,102.0,"[13/4.0, 16/4.0, 17/2.0]","[P1, P4]","[102.0, 126.0, 130.0]","[1, -3, 1, 2, 1, -3, 2, 2]","[24.0, 4.0]","[Altus, Tenor, Cantus]",FUGA,3,True,P1_P4,24.0_4.0,1_-3_1_2,P1_P4_24.0_4.0_1_-3_1_2,P1_P4_24.0_4.0,P1_P4_1_-3_1_2
242,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,152.0,"[20/1.0, 21/2.0, 23/4.0, 26/2.0, 28/4.0, 31/2.0]","[P5, P4, P-12, M9, P4]","[152.0, 162.0, 182.0, 202.0, 222.0, 242.0]","[1, -3, 3, -2]","[10.0, 20.0, 20.0, 20.0, 20.0]","[Bassus, Altus, Cantus, Bassus, Tenor, Cantus]",FUGA,6,False,P5_P4_P-12_M9_P4,10.0_20.0_20.0_20.0_20.0,1_-3_3_-2,P5_P4_P-12_M9_P4_10.0_20.0_20.0_20.0_20.0_1_-3...,P5_P4_P-12_M9_P4_10.0_20.0_20.0_20.0_20.0,P5_P4_P-12_M9_P4_1_-3_3_-2
243,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,404.0,"[50/3.0, 53/3.0]",[P-8],"[404.0, 428.0]","[1, 1, -3, 3]",[24.0],"[Cantus, Quinta Pars]",FUGA,2,False,P-8,24.0,1_1_-3_3,P-8_24.0_1_1_-3_3,P-8_24.0,P-8_1_1_-3_3


In [17]:
filtered = final.loc[final['Number_Entries'] < 20] 
filtered

,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,MINT,TINT,SOG,ALL,ALL_INT,ALL_SOG
0,Antoine de Févin,Missa Ave Maria: Credo,0.0,"[1/1.0, 3/3.0]",[P8],"[0.0, 20.0]","[4, 1, 1, 2, 4, 1, 2, 2]",[20.0],"[Altus, Sup[erius]]",FUGA,2,True,P8,20.0,4_1_1_2,P8_20.0_4_1_1_2,P8_20.0,P8_4_1_1_2
1,Antoine de Févin,Missa Ave Maria: Credo,30.0,"[4/4.0, 7/2.0]",[P8],"[30.0, 50.0]","[4, 1, -2, -2]",[20.0],"[Altus, Sup[erius]]",FUGA,2,False,P8,20.0,4_1_-2_-2,P8_20.0_4_1_-2_-2,P8_20.0,P8_4_1_-2_-2
2,Antoine de Févin,Missa Ave Maria: Credo,90.0,"[12/2.0, 12/4.0]",[m6],"[90.0, 94.0]","[1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1]",[4.0],"[Altus, Sup[erius]]",FUGA,2,True,m6,4.0,1_1_3_1,m6_4.0_1_1_3_1,m6_4.0,m6_1_1_3_1
3,Antoine de Févin,Missa Ave Maria: Credo,118.0,"[15/4.0, 16/2.0]",[P5],"[118.0, 122.0]","[2, 2, 2, 2, 2, 2, 2, 1]",[4.0],"[Altus, Sup[erius]]",FUGA,2,True,P5,4.0,2_2_2_2,P5_4.0_2_2_2_2,P5_4.0,P5_2_2_2_2
4,Antoine de Févin,Missa Ave Maria: Credo,156.0,"[20/3.0, 27/2.0, 27/4.0]","[m-7, P5]","[156.0, 210.0, 214.0]","[1, 2, 2, -2, 2, 2, 2, -2]","[54.0, 4.0]","[Tenor, Bassus, Tenor]",FUGA,3,True,m-7_P5,54.0_4.0,1_2_2_-2,m-7_P5_54.0_4.0_1_2_2_-2,m-7_P5_54.0_4.0,m-7_P5_1_2_2_-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,0.0,"[1/1.0, 2/1.0, 5/3.0, 7/3.0, 8/4.0]","[P-5, P-4, P-5, P8]","[0.0, 8.0, 36.0, 52.0, 62.0]","[-3, 3, 2, -2, -3, 2, 2, -2]","[8.0, 28.0, 16.0, 10.0]","[Cantus, Altus, Tenor, Bassus, Altus]",FUGA,5,True,P-5_P-4_P-5_P8,8.0_28.0_16.0_10.0,-3_3_2_-2,P-5_P-4_P-5_P8_8.0_28.0_16.0_10.0_-3_3_2_-2,P-5_P-4_P-5_P8_8.0_28.0_16.0_10.0,P-5_P-4_P-5_P8_-3_3_2_-2
241,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,102.0,"[13/4.0, 16/4.0, 17/2.0]","[P1, P4]","[102.0, 126.0, 130.0]","[1, -3, 1, 2, 1, -3, 2, 2]","[24.0, 4.0]","[Altus, Tenor, Cantus]",FUGA,3,True,P1_P4,24.0_4.0,1_-3_1_2,P1_P4_24.0_4.0_1_-3_1_2,P1_P4_24.0_4.0,P1_P4_1_-3_1_2
242,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,152.0,"[20/1.0, 21/2.0, 23/4.0, 26/2.0, 28/4.0, 31/2.0]","[P5, P4, P-12, M9, P4]","[152.0, 162.0, 182.0, 202.0, 222.0, 242.0]","[1, -3, 3, -2]","[10.0, 20.0, 20.0, 20.0, 20.0]","[Bassus, Altus, Cantus, Bassus, Tenor, Cantus]",FUGA,6,False,P5_P4_P-12_M9_P4,10.0_20.0_20.0_20.0_20.0,1_-3_3_-2,P5_P4_P-12_M9_P4_10.0_20.0_20.0_20.0_20.0_1_-3...,P5_P4_P-12_M9_P4_10.0_20.0_20.0_20.0_20.0,P5_P4_P-12_M9_P4_1_-3_3_-2
243,Giovanni Pierluigi da Palestrina,Missa Veni sponsa Christi: Agnus Dei,404.0,"[50/3.0, 53/3.0]",[P-8],"[404.0, 428.0]","[1, 1, -3, 3]",[24.0],"[Cantus, Quinta Pars]",FUGA,2,False,P-8,24.0,1_1_-3_3,P-8_24.0_1_1_-3_3,P-8_24.0,P-8_1_1_-3_3


### Network Visualization with Louvain Communities

In [18]:

f2 = filtered.groupby('MINT')['Title'].apply(list).reset_index()
f2


,MINT,Title
0,M-2,[Missa Quo abiit dilectus tuus: Gloria]
1,M-2_P-4_P12_M-9,[Missa Vidi speciosam: Gloria]
2,M-6,[Missa Ave Maria: Credo]
3,M-6_P-8,[Missa Vidi speciosam: Credo]
4,M-6_m-3_M-6_M13_P-5,[Missa Ave Maria: Credo]
...,...,...
151,m-7,[Quo abiit dilectus tuus]
152,m-7_P5,[Missa Ave Maria: Credo]
153,m3,[Missa Veni sponsa Christi: Credo]
154,m6,[Missa Ave Maria: Credo]


In [19]:
pairs = f2.Title.apply(lambda x: list(combinations(x, 2)))
pairs

0      []
1      []
2      []
3      []
4      []
       ..
151    []
152    []
153    []
154    []
155    []
Name: Title, Length: 156, dtype: object

In [20]:
pairs2 = pairs.explode().dropna()
pairs2

13        (Veni speciosam, Missa Vidi speciosam: Gloria)
13     (Veni speciosam, Missa Quo abiit dilectus tuus...
13     (Veni speciosam, Missa Quo abiit dilectus tuus...
13     (Veni speciosam, Missa Quo abiit dilectus tuus...
13     (Veni speciosam, Missa Quo abiit dilectus tuus...
                             ...                        
143    (Missa Ave Maria: Sanctus, Missa Vidi speciosa...
143    (Missa Vidi speciosam: Credo, Missa Vidi speci...
143    (Missa Vidi speciosam: Credo, Missa Vidi speci...
143    (Missa Vidi speciosam: Credo, Missa Vidi speci...
149    (Missa Vidi speciosam: Credo, Missa Veni spons...
Name: Title, Length: 465, dtype: object

In [21]:
unique_pairs = pairs.explode().dropna().unique()
unique_pairs

array([('Veni speciosam', 'Missa Vidi speciosam: Gloria'),
       ('Veni speciosam', 'Missa Quo abiit dilectus tuus: Gloria'),
       ('Veni speciosam', 'Missa Quo abiit dilectus tuus: Credo'),
       ('Veni speciosam', 'Missa Veni sponsa Christi: Credo'),
       ('Missa Vidi speciosam: Gloria', 'Missa Quo abiit dilectus tuus: Gloria'),
       ('Missa Vidi speciosam: Gloria', 'Missa Quo abiit dilectus tuus: Credo'),
       ('Missa Vidi speciosam: Gloria', 'Missa Veni sponsa Christi: Credo'),
       ('Missa Quo abiit dilectus tuus: Gloria', 'Missa Quo abiit dilectus tuus: Gloria'),
       ('Missa Quo abiit dilectus tuus: Gloria', 'Missa Quo abiit dilectus tuus: Credo'),
       ('Missa Quo abiit dilectus tuus: Gloria', 'Missa Veni sponsa Christi: Credo'),
       ('Missa Quo abiit dilectus tuus: Credo', 'Missa Quo abiit dilectus tuus: Credo'),
       ('Missa Quo abiit dilectus tuus: Credo', 'Missa Veni sponsa Christi: Credo'),
       ('Quo abiit dilectus tuus', 'Missa Quo abiit dilectus t

In [22]:
pd.Series(unique_pairs).isna().sum()

0

In [23]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

In [24]:
pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="white", font_color="black")

In [25]:
G = nx.Graph()
G.add_edges_from(unique_pairs)
G = add_communities(G)

In [26]:
pyvis_graph.from_nx(G)

In [27]:
pyvis_graph.show('MINT.html')


#### Below is Development Work

In [ ]:
filtered = output.loc[output['Number_Entries'] < 4] 
filtered

In [ ]:
output = output.loc[output['Presentation_Type'] == "PEN"] 
output

In [ ]:
offset_diffs = [2.0, 1.0, 2.0, 3.0, 5.0, 6.0]
# some_list[start:stop:step]
alt_list = offset_diffs[::2]
alt_list

In [ ]:
# this works with ONE list of offsets

points2 = pd.DataFrame()
split_list = [90.0, 94.0, 102.0, 106.0, 134.0, 146.0, 162.0]

l = len(split_list)  
for r in range(3, l):
    list_combinations = list(combinations(split_list, r))
#             combo_time_ints = []
    for combo in list_combinations:
        combo_time_ints = numpy.diff(combo).tolist()
        combo_array = entry_array[entry_array.index.get_level_values(0).isin(combo)]
        combo_voice_list = combo_array['voice'].to_list()
        combo_patterns = combo_array['pattern']
        unique_combo_patterns = list(set(combo_patterns))
        tone_coordinates =  list(zip(combo, combo_voice_list))
# tone_coordinates.ffill(inplace=True)
        mel_ints = find_entry_int_distance(tone_coordinates, piece)
        hidden_type = classify_by_offset(combo_time_ints)

        meas_beat = det[det.index.get_level_values('Offset').isin(combo)]
        mb2 = meas_beat.reset_index()
        mb2['mb'] = mb2["Measure"].astype(str) + "/" + mb2["Beat"].astype(str)
        meas_beat_list = mb2['mb'].to_list()

        combo_temp = {'First_Offset': combo[0], 
            'Offsets': combo, 
            'Measures_Beats': meas_beat_list,
            'Presentation_Type': hidden_type,
            "Soggetti": unique_combo_patterns,
            'Voices': combo_voice_list, 
            'Time_Entry_Intervals': combo_time_ints, 
            'Melodic_Entry_Intervals': mel_ints}

        if 'PEN' in hidden_type:
            points2 = points2.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#             points2 = points2[points2['Offsets'].apply(len) > 1]
        if 'ID' in hidden_type:
            points2 = points2.append(combo_temp, ignore_index=True).sort_values("First_Offset")
#             points2 = points2[points2['Offsets'].apply(len) > 1]
        
        
# combo_time_ints
# combo_array
# # combo_voice_list
# # combo_patterns
# # unique_combo_patterns
# # tone_coordinates
# # mel_ints
# # combo_temp
points2

In [ ]:
# this finds hidden fugas.  
# try to run each of the first set of results above ('points') through this tool, then append the 
# new results to the full DF, and sort again.  
# mark each long pattern with 'has hidden pattern' boolean?  or ?

sample_list = points["Offsets"][4]

hidden_pts = []
n = len(sample_list)
for item in range(3, n):
    list_combinations = list(combinations(sample_list, item))
    for group in list_combinations:
        group_time_ints = numpy.diff(group).tolist()
        hidden_type = classify_by_offset(group_time_ints)
        if 'PEN' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        if 'ID' in hidden_type:
            print(group)
            print(group_time_ints)
            print(hidden_type)
            hidden_pts.append(group_time_ints)
        

list_combinations

In [ ]:
def classify_entries_as_presentation_types(piece):
    # Classifier with Functions
    points = pd.DataFrame()
    points2 = pd.DataFrame()
    # new_offset_list = []
    nr = piece.getNoteRest()
    det = piece.detailIndex(nr, offset=True)

    # durations and ngrams of durations
    dur = piece.getDuration(df=nr)
    dur_ng = piece.getNgrams(df=dur, n=4)

    # ngrams of melodic entries
    # for chromatic, use:
    # piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
    mel = piece.getMelodicEntries(n=4)
    mels_stacked = mel.stack().to_frame()
    mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

    # edit distance, based on side-by-side comparison of melodic ngrams
    # gets flexed and other similar soggetti
    dist = piece.getDistance(mel)
    dist_stack = dist.stack().to_frame()


    # filter distances to threshold.  <2 is good
    filtered_dist_stack = dist_stack[dist_stack[0] < 2]
    filtered_dist = filtered_dist_stack.reset_index()
    filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

    # Group the filtered distanced patterns
    full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

    for matches in full_list_of_matches["match"]:
        related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
        entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
        offset_list = entry_array.index.to_list()
        split_list = list(split_by_threshold(offset_list))
        # here is the list of starting offsets of the original set of entries:  slist
        slist = split_list[0]
        temp = temp_dict_of_details(slist, entry_array, det, matches)

        points = points.append(temp, ignore_index=True)
        points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
        points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
        points = points[points['Offsets'].apply(len) > 1]

        l = len(slist)
        if l > 2:
            for r in range(3, l):
    #             list_combinations = list(combinations(slist, r))
                list_combinations = list(combinations(slist, r))
                for slist in list_combinations:

                    temp = temp_dict_of_details(slist, entry_array, det, matches)

                    temp["Presentation_Type"] = classify_by_offset(temp['Time_Entry_Intervals'])

                    if 'PEN' in temp["Presentation_Type"]:
                        points2 = points2.append(temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                        points2 = points2[points2['Offsets'].apply(len) > 1]
                    if 'ID' in temp["Presentation_Type"]:
                        points2 = points2.append(combo_temp, ignore_index=True)#.sort_values("First_Offset")
    #                     points = points.append(combo_temp, ignore_index=True).sort_values("First_Offset")
                points2.sort_values("First_Offset")
                points2.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)

    points_combined = points.append(points2, ignore_index=True).sort_values("First_Offset").reset_index(drop=True)
    points_combined['Flexed_Entries'] = points_combined["Soggetti"].apply(len) > 1
    points_combined["Number_Entries"] = points["Offsets"].apply(len)     
    return points2


In [ ]:
# This test works

points = pd.DataFrame()
points2 = pd.DataFrame()
# new_offset_list = []
nr = piece.getNoteRest()
det = piece.detailIndex(nr, offset=True)

# durations and ngrams of durations
dur = piece.getDuration(df=nr)
dur_ng = piece.getNgrams(df=dur, n=4)

# ngrams of melodic entries
# for chromatic, use:
# piece.getMelodicEntries(interval_settings=('c', True, True), n=5)
mel = piece.getMelodicEntries(n=4)
mels_stacked = mel.stack().to_frame()
mels_stacked.rename(columns =  {0:"pattern"}, inplace = True)

# edit distance, based on side-by-side comparison of melodic ngrams
# gets flexed and other similar soggetti
dist = piece.getDistance(mel)
dist_stack = dist.stack().to_frame()


# filter distances to threshold.  <2 is good
filtered_dist_stack = dist_stack[dist_stack[0] < 2]
filtered_dist = filtered_dist_stack.reset_index()
filtered_dist.rename(columns =  {'level_0':"source", 'level_1':'match'}, inplace = True)

# Group the filtered distanced patterns
full_list_of_matches = filtered_dist.groupby('source')['match'].apply(list).reset_index()

for matches in full_list_of_matches["match"]:
    related_entry_list = mels_stacked[mels_stacked['pattern'].isin(matches)]
    entry_array = related_entry_list.reset_index(level=1).rename(columns = {'level_1': "voice", 0: "pattern"})
    offset_list = entry_array.index.to_list()
    split_list = list(split_by_threshold(offset_list))
    # here is the list of starting offsets of the original set of entries:  slist
    slist = split_list[0]
    temp = temp_dict_of_details(slist, entry_array, det, matches)

    points = points.append(temp, ignore_index=True)
    points['Presentation_Type'] = points['Time_Entry_Intervals'].apply(classify_by_offset)
    points.drop_duplicates(subset=["First_Offset"], keep='first', inplace = True)
    points = points[points['Offsets'].apply(len) > 1]

    test = [278.0, 286.0, 294.0, 298.0, 306.0, 310.0]

    l = len(test)  
    for item in range(3, l):
        list_combinations = list(combinations(test, item))
        for group in list_combinations:
            group_time_ints = numpy.diff(group).tolist()
            hidden_type = classify_by_offset(group_time_ints)
            for item in group:
    #         print(item)
                array = group[entry_array.index.get_level_values(0).isin(item)]
                short_offset_list = array.index.to_list()
                time_ints = numpy.diff(array.index).tolist()
                voice_list = array['voice'].to_list()
                if 'PEN' in hidden_type:
                    print(group)
                    print(group_time_ints)
                    print(hidden_type)
                    hidden_pts.append(group_time_ints)
                if 'ID' in hidden_type:
                    print(group)
                    print(group_time_ints)
                    print(hidden_type)
                    hidden_pts.append(group_time_ints)
# len(split_list[0])           

In [ ]:
#  This shows how the classifier works:

if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
    print('This is a PEN')
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
    print('This is an ID')
elif len(offset_diffs) >= 1:
    print('This is a FUGA')

In [ ]:
# This shows how combinations works for a given set of time intervals
offset_diffs = [12.0, 32.0, 12.0, 4.0]
l = len(offset_diffs)
# print(l)
if l > 2:
    for r in range(3, l):
        print(r)
        list_combinations = list(combinations(offset_diffs, r))
#         for slist in list_combinations:
        print(list_combinations)

In [ ]:
slist = [278.0, 286.0, 294.0, 298.0, 306.0, 310.0]
l = len(slist)
# for r in range(3, 6):
list_combinations = list(combinations(slist, 4))
#     for tiny_list in list_combinations:

In [ ]:
print(list_combinations)

In [ ]:
list_offsets = [294.0, 298.0, 306.0, 310.0]

In [ ]:
offset_diffs = [4, 5, 6]

In [ ]:
alt_list = offset_diffs[::2]

if len(set(offset_diffs)) == 1 and len(offset_diffs) > 1:
    print('This is a PEN')
    # elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
elif (len(offset_diffs) % 2 != 0) and (len(set(alt_list)) == 1) and (len(offset_diffs) >= 3):
    print('This is an ID')
elif len(offset_diffs) >= 1:
    print('This is a FUGA')